# Content Based Recommendation System


So, I am going to use TF-IDF  technique to make two different recommendation models with sigmoid kernel fucntion and one with linear cosine similarity and let's see which one is better.
****

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-shows/netflix_titles.csv


In [2]:
data = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
print (data.shape)
data.head(5)

(7787, 12)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   object
 10  listed_in     7787 non-null   object
 11  description   7787 non-null   object
dtypes: int64(1), object(11)
memory usage: 730.2+ KB


**1. Content based recommendation system using sigmoid kernel to find the similarities**

****I am dividing 2 different sets one for movie recommendations and one for TV/Shows. Also I am using a combined function to compute the similarity****

# Movie set* ****

In [4]:
movies = data[data['type']=='Movie'].reset_index()
movies = movies.drop(columns = ['duration','country','date_added','release_year','show_id','type','index','listed_in'])

In [5]:
movies['director'] = movies['director'].fillna("")
movies['cast'] = movies['cast'].fillna("")
movies['combined'] = movies['description']+movies['cast']+movies['director']
movies.head(5)

,title,director,cast,rating,description,combined
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",TV-MA,After a devastating earthquake hits Mexico Cit...,After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",R,"When an army recruit is found dead, his fellow...","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",PG-13,"In a postapocalyptic world, rag-doll robots hi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",PG-13,A brilliant group of students become card-coun...,A brilliant group of students become card-coun...
4,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",TV-MA,"After an awful accident, a couple admitted to ...","After an awful accident, a couple admitted to ..."


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')

In [7]:
movies['combined'] = movies['combined'].fillna("")

In [8]:
tfv_matrix = tfv.fit_transform(movies['combined'])
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix,tfv_matrix)
sig[0]
indices = pd.Series(movies.index,index = movies['title']).drop_duplicates()
indices

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


title
7:19                                          0
23:59                                         1
9                                             2
21                                            3
122                                           4
                                           ... 
Zoom                                       5372
Zozo                                       5373
Zubaan                                     5374
Zulu Man in Japan                          5375
ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS    5376
Length: 5377, dtype: int64

In [9]:
def recommend(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    movies_indices = [i[0] for i in sig_scores]
    return movies['title'].iloc[movies_indices]

In [10]:
recommend('Zulu Man in Japan')

2349                Kevin Hart: Seriously Funny
5230                What Would Sophia Loren Do?
376                       An American in Madras
1598                               French Toast
3573                               Reincarnated
4264                            The Bittersweet
2257            Joshua: Teenager vs. Superpower
965      Chingo Bling: They Can't Deport Us All
4214    The Adventures of Sharkboy and Lavagirl
4773                       The Skin of the Wolf
Name: title, dtype: object

In [11]:
recommend('Zubaan')

4311                        The Cakemaker
3917              Silent Hill: Revelation
2239                          John & Jane
1777                               Guilty
587                           Be Somebody
4252    The Battered Bastards of Baseball
1566                            Forgotten
1234          Dieter Nuhr: Nuhr in Berlin
731                        Bleed for This
3730           Sam Kinison: Live in Vegas
Name: title, dtype: object

In [12]:
recommend('The Cakemaker')

5374                             Zubaan
3917            Silent Hill: Revelation
1456                           Extremis
3016              My Life as a Zucchini
3724                            Saladin
2835             Michael Lost and Found
1408                              Enemy
3755    Sarah Silverman A Speck of Dust
5076                     Under The Skin
2704                             Maktub
Name: title, dtype: object

# TV Shows set

In [13]:
shows = data[data['type']=='TV Show'].reset_index()
shows = shows.drop(columns = ['duration','country','date_added','release_year','show_id','type','index','listed_in'])

In [14]:
shows['director'] = shows['director'].fillna("")
shows['cast'] = shows['cast'].fillna("")
shows['combined'] = shows['cast']+shows['director']+shows['description']

In [15]:
shows.head()

,title,director,cast,rating,description,combined
0,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,In a future where the elite inhabit an island ...,"João Miguel, Bianca Comparato, Michel Gomes, R..."
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",TV-MA,A genetics professor experiments with a treatm...,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan..."
2,1983,,"Robert Więckiewicz, Maciej Musiał, Michalina O...",TV-MA,"In this dark alt-history thriller, a naïve law...","Robert Więckiewicz, Maciej Musiał, Michalina O..."
3,1994,Diego Enrique Osorno,,TV-MA,Archival video and new interviews examine Mexi...,Diego Enrique OsornoArchival video and new int...
4,Feb-09,,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",TV-14,"As a psychology professor faces Alzheimer's, h...","Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-..."


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(min_df = 3,max_features = None,analyzer = 'word',token_pattern = 'r\w{1,}', ngram_range = (1,3), stop_words = 'english')
shows['combined'] = shows['combined'].fillna("")

In [17]:
tfv_matrix_shows = tfv.fit_transform(shows['combined'])
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix_shows,tfv_matrix_shows)
sig[0]
indices = pd.Series(shows.index,index = shows['title']).drop_duplicates()
indices

/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ral', 'rd', 'rds', 'ready', 'reafter', 'reas', 'reby', 'red', 'ree', 'refore', 'rehand', 'rein', 'reover', 'rest', 'reupon', 'rever', 'rhaps', 'ribe', 'ring', 'rious', 'rly', 'rmer', 'rmerly', 'rom', 'ront', 'ross', 'rough', 'roughout', 'round', 'rs', 'rself', 'rselves', 'rst', 'rt', 'rtheless', 'rther', 'rty', 'ru', 'rwards', 'rwise', 'ry', 'ryone', 'rything', 'rywhere'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


title
3%                          0
46                          1
1983                        2
1994                        3
Feb-09                      4
                         ... 
Zindagi Gulzar Hai       2405
Zoids Wild               2406
Zombie Dumb              2407
Zona Rosa                2408
Zumbo's Just Desserts    2409
Length: 2410, dtype: int64

In [18]:
def recommend(title,sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores,key = lambda x:x[1], reverse = True)
    sig_scores = sig_scores[1:11]
    shows_indices = [i[0] for i in sig_scores]
    return shows['title'].iloc[shows_indices]

In [19]:
recommend("Friends")

706                       Genius of the Modern World
418                 Comedians in Cars Getting Coffee
1960               The goop lab with Gwyneth Paltrow
1221                Melodies of Life - Born This Way
959                                       JingleKids
1777    Strawberry Shortcake: Berry Bitty Adventures
2285                        VeggieTales in the House
1640                                         Scandal
1279                 Monty Python Best Bits (mostly)
1282                    Monty Python's Flying Circus
Name: title, dtype: object

In [20]:
recommend("Crash Landing on You")

54                  Abnormal Summit
1843    That Winter, the Wind Blows
211                          Betaal
1247             Million Pound Menu
1008              Kim's Convenience
890                       I'm Sorry
700                Game Winning Hit
2284        VeggieTales in the City
1466                        Persona
367              Chicago Typewriter
Name: title, dtype: object

**Personally , I don't feel that these recommendations are good enough to consider. So, I am doing the same method but this time using Cosine similarities with linear kernel**

# Movies set* 

In [21]:
movies.head(5)

,title,director,cast,rating,description,combined
0,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",TV-MA,After a devastating earthquake hits Mexico Cit...,After a devastating earthquake hits Mexico Cit...
1,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",R,"When an army recruit is found dead, his fellow...","When an army recruit is found dead, his fellow..."
2,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",PG-13,"In a postapocalyptic world, rag-doll robots hi...","In a postapocalyptic world, rag-doll robots hi..."
3,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",PG-13,A brilliant group of students become card-coun...,A brilliant group of students become card-coun...
4,122,Yasir Al Yasiri,"Amina Khalil, Ahmed Dawood, Tarek Lotfy, Ahmed...",TV-MA,"After an awful accident, a couple admitted to ...","After an awful accident, a couple admitted to ..."


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')


movies['combined'] = movies['combined'].fillna('')
tfidf_matrix = tfidf.fit_transform(movies['combined'])


tfidf_matrix.shape

(5377, 40215)

In [23]:
tfidf.get_feature_names()[5000:5010]

['brauenkaryn',
 'braugher',
 'braun',
 'braunjon',
 'braunschweig',
 'braunschweigethan',
 'braunstein',
 'bravado',
 'bravani',
 'brave']

In [24]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(5377, 5377)

In [25]:
cosine_sim[1]

array([0., 1., 0., ..., 0., 0., 0.])

In [26]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()
indices

title
7:19                                          0
23:59                                         1
9                                             2
21                                            3
122                                           4
                                           ... 
Zoom                                       5372
Zozo                                       5373
Zubaan                                     5374
Zulu Man in Japan                          5375
ZZ TOP: THAT LITTLE OL' BAND FROM TEXAS    5376
Length: 5377, dtype: int64

In [27]:
def recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]


In [28]:
recommendations('Zulu Man in Japan')

378            An American Tail: Fievel Goes West
2342          Keith Richards: Under the Influence
4979                                Trainspotting
5045                         Tyke Elephant Outlaw
1400    Emicida: AmarElo - It’s All For Yesterday
965        Chingo Bling: They Can't Deport Us All
3025                           My Octopus Teacher
1663         GEN HOSHINO STADIUM TOUR “POP VIRUS”
3842            Seth Rogen's Hilarity for Charity
2788                                      Maynard
Name: title, dtype: object

In [29]:
recommendations("Zubaan")

510                                       Baazaar
2176                             Jatts in Golmaal
4252            The Battered Bastards of Baseball
2302                                     Kajraare
1435                              Evening Shadows
421                       Ankur Arora Murder Case
3745                                        Sanju
2421    Krish Trish and Baltiboy: Face Your Fears
3027                                   My Own Man
1640                                         Game
Name: title, dtype: object

In [30]:
recommendations("Sanju")

3399                                         PK
2666                               Lust Stories
433                                       Anwar
4873                            The Zoya Factor
1844                       Haseena Maan Jaayegi
1222                         Dharam Sankat Mein
3378                Phir Bhi Dil Hai Hindustani
1259    Do Paise Ki Dhoop Chaar Aane Ki Baarish
1001                               Chup Chup Ke
1691                          Ghulam-E-Musthafa
Name: title, dtype: object

**My experience can vote for this recommendation model better than the first one**

# TV Shows set

In [31]:
shows.head(5)

,title,director,cast,rating,description,combined
0,3%,,"João Miguel, Bianca Comparato, Michel Gomes, R...",TV-MA,In a future where the elite inhabit an island ...,"João Miguel, Bianca Comparato, Michel Gomes, R..."
1,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",TV-MA,A genetics professor experiments with a treatm...,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan..."
2,1983,,"Robert Więckiewicz, Maciej Musiał, Michalina O...",TV-MA,"In this dark alt-history thriller, a naïve law...","Robert Więckiewicz, Maciej Musiał, Michalina O..."
3,1994,Diego Enrique Osorno,,TV-MA,Archival video and new interviews examine Mexi...,Diego Enrique OsornoArchival video and new int...
4,Feb-09,,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",TV-14,"As a psychology professor faces Alzheimer's, h...","Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-..."


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')


shows['combined'] = shows['combined'].fillna('')
tfidf_matrix = tfidf.fit_transform(shows['combined'])


tfidf_matrix.shape

(2410, 23998)

In [33]:
tfidf.get_feature_names()[5000:5010]

['damage',
 'damata',
 'damayanti',
 'damgacıafter',
 'damian',
 'damidotafter',
 'damien',
 'damijo',
 'damilola',
 'damián']

In [34]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(2410, 2410)

In [35]:
cosine_sim[1]

array([0., 1., 0., ..., 0., 0., 0.])

In [36]:
indices = pd.Series(shows.index, index=shows['title']).drop_duplicates()
indices

title
3%                          0
46                          1
1983                        2
1994                        3
Feb-09                      4
                         ... 
Zindagi Gulzar Hai       2405
Zoids Wild               2406
Zombie Dumb              2407
Zona Rosa                2408
Zumbo's Just Desserts    2409
Length: 2410, dtype: int64

In [37]:
def recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    shows_indices = [i[0] for i in sim_scores]
    return shows['title'].iloc[shows_indices]

In [38]:
recommendations("Friends")

598                  Episodes
1182          Man with a Plan
1092               Life Story
260           BoJack Horseman
447            Criminal Minds
959                JingleKids
752               Gossip Girl
248             Blood of Zeus
524     Dinotrux Supercharged
516                 Diagnosis
Name: title, dtype: object

In [39]:
recommendations("Crash Landing on You")

277                 Boys Over Flowers
119                   Another Miss Oh
911                        Inheritors
940          It's Okay to Not Be Okay
2007        The King: Eternal Monarch
2313                     Was It Love?
862                 Hospital Playlist
198     Because This Is My First Life
1272                          MONKART
40                A Love So Beautiful
Name: title, dtype: object

In [40]:
recommendations("It's Okay to Not Be Okay")

435     Crash Landing on You
2313            Was It Love?
2030         The Lies Within
1649           Secret Affair
911               Inheritors
933                     Iris
862        Hospital Playlist
1614                  Run On
164      Bad Guys: Vile City
878                    Hyena
Name: title, dtype: object

**My Kdrama addicted brain is absolutely in favour of these recommendations more than the first one**

**# So, overall I would say that the 2nd one is better. I would also try to make this better in future.** Thank you for viewing this notebook😊😊